# ETL pipeline using data from 3 web APIs

In [1]:
import arcpy
import requests
import zipfile
import io
import geopandas as gpd 
import pandas as pd
import json
from shapely import geometry
from fiona.crs import from_epsg
import fiona
from urllib.request import urlretrieve as retrieve
import csv

## Minnesota Gespatial Commons
Two datasets about the geological land type associations and location of springs are downloaded from this portal.


In [2]:
# The zip files are downloaded from the web portal and extracted to the local directory

# Geological land type associations dataset
land_type_link = r'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/geos_land_type_associations/shp_geos_land_type_associations.zip'
land_type_output = requests.post(land_type_link)
unzip_land_type = zipfile.ZipFile(io.BytesIO(land_type_output.content)).extractall(r'E:\ArcGIS_1\Lab1\Lab1_API')

# Springs dataset
springs_link = r'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/env_mn_springs_inventory/shp_env_mn_springs_inventory.zip'
springs_output = requests.post(springs_link)
unzip_springs = zipfile.ZipFile(io.BytesIO(springs_output.content)).extractall(r'E:\ArcGIS_1\Lab1\Lab1_API')

In [3]:
# Projects the shapefiles from NAD 1983 UTM Zone 15N to WGS 1984 UTM Zone 15N
arcpy.management.Project(r"E:\ArcGIS_1\Lab1\Lab1_API\ecs_land_type_associations_of_mn.shp", r"E:\ArcGIS_1\Lab1\Lab1_API\LandType_Project.shp", "PROJCS['WGS_1984_UTM_Zone_15N',GEOGCS['GCS_WGS_1984',DATUM['D_WGS_1984',SPHEROID['WGS_1984',6378137.0,298.257223563]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]],PROJECTION['Transverse_Mercator'],PARAMETER['False_Easting',500000.0],PARAMETER['False_Northing',0.0],PARAMETER['Central_Meridian',-93.0],PARAMETER['Scale_Factor',0.9996],PARAMETER['Latitude_Of_Origin',0.0],UNIT['Meter',1.0]]", "WGS_1984_(ITRF00)_To_NAD_1983", "PROJCS['NAD_1983_UTM_Zone_15N',GEOGCS['GCS_North_American_1983',DATUM['D_North_American_1983',SPHEROID['GRS_1980',6378137.0,298.257222101]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]],PROJECTION['Transverse_Mercator'],PARAMETER['False_Easting',500000.0],PARAMETER['False_Northing',0.0],PARAMETER['Central_Meridian',-93.0],PARAMETER['Scale_Factor',0.9996],PARAMETER['Latitude_Of_Origin',0.0],UNIT['Meter',1.0]]", "NO_PRESERVE_SHAPE", None, "NO_VERTICAL")
arcpy.management.Project(r"E:\ArcGIS_1\Lab1\Lab1_API\springs.shp", r"E:\ArcGIS_1\Lab1\Lab1_API\Springs_Project.shp", "PROJCS['WGS_1984_UTM_Zone_15N',GEOGCS['GCS_WGS_1984',DATUM['D_WGS_1984',SPHEROID['WGS_1984',6378137.0,298.257223563]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]],PROJECTION['Transverse_Mercator'],PARAMETER['False_Easting',500000.0],PARAMETER['False_Northing',0.0],PARAMETER['Central_Meridian',-93.0],PARAMETER['Scale_Factor',0.9996],PARAMETER['Latitude_Of_Origin',0.0],UNIT['Meter',1.0]]", "WGS_1984_(ITRF00)_To_NAD_1983", "PROJCS['NAD_1983_UTM_Zone_15N',GEOGCS['GCS_North_American_1983',DATUM['D_North_American_1983',SPHEROID['GRS_1980',6378137.0,298.257222101]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]],PROJECTION['Transverse_Mercator'],PARAMETER['False_Easting',500000.0],PARAMETER['False_Northing',0.0],PARAMETER['Central_Meridian',-93.0],PARAMETER['Scale_Factor',0.9996],PARAMETER['Latitude_Of_Origin',0.0],UNIT['Meter',1.0]]", "NO_PRESERVE_SHAPE", None, "NO_VERTICAL")

<Result 'E:\\ArcGIS_1\\Lab1\\Lab1_API\\Springs_Project.shp'>

In [4]:
# Spatial join
arcpy.analysis.SpatialJoin("E:\ArcGIS_1\Lab1\Lab1_API\Springs_Project.shp", "E:\ArcGIS_1\Lab1\Lab1_API\LandType_Project.shp", r"E:\ArcGIS_1\Lab1\Lab1_API\Springs_SpatialJoin_LandType.shp", "JOIN_ONE_TO_ONE", "KEEP_ALL", 'survey_nam "survey_nam" true true false 50 Text 0 0,First,#,Springs_Project,survey_nam,0,50;name "name" true true false 110 Text 0 0,First,#,Springs_Project,name,0,110;feature "feature" true true false 1 Text 0 0,First,#,Springs_Project,feature,0,1;feature_su "feature_su" true true false 10 Text 0 0,First,#,Springs_Project,feature_su,0,10;feature_ar "feature_ar" true true false 10 Text 0 0,First,#,Springs_Project,feature_ar,0,10;spring_typ "spring_typ" true true false 50 Text 0 0,First,#,Springs_Project,spring_typ,0,50;spring_t_1 "spring_t_1" true true false 250 Text 0 0,First,#,Springs_Project,spring_t_1,0,250;lithology "lithology" true true false 50 Text 0 0,First,#,Springs_Project,lithology,0,50;lith_comme "lith_comme" true true false 200 Text 0 0,First,#,Springs_Project,lith_comme,0,200;mineral_pr "mineral_pr" true true false 15 Text 0 0,First,#,Springs_Project,mineral_pr,0,15;min_prec_c "min_prec_c" true true false 250 Text 0 0,First,#,Springs_Project,min_prec_c,0,250;photo "photo" true true false 1 Text 0 0,First,#,Springs_Project,photo,0,1;flow_measu "flow_measu" true true false 5 Text 0 0,First,#,Springs_Project,flow_measu,0,5;field_meas "field_meas" true true false 5 Text 0 0,First,#,Springs_Project,field_meas,0,5;chemistry_ "chemistry_" true true false 5 Text 0 0,First,#,Springs_Project,chemistry_,0,5;field_chec "field_chec" true true false 5 Text 0 0,First,#,Springs_Project,field_chec,0,5;field_ch_1 "field_ch_1" true true false 8 Date 0 0,First,#,Springs_Project,field_ch_1,-1,-1;gcm_code "gcm_code" true true false 3 Text 0 0,First,#,Springs_Project,gcm_code,0,3;general_co "general_co" true true false 254 Text 0 0,First,#,Springs_Project,general_co,0,254;flowing "flowing" true true false 1 Text 0 0,First,#,Springs_Project,flowing,0,1;flow "flow" true true false 8 Double 0 0,First,#,Springs_Project,flow,-1,-1;flow_units "flow_units" true true false 3 Text 0 0,First,#,Springs_Project,flow_units,0,3;flow_mc "flow_mc" true true false 10 Text 0 0,First,#,Springs_Project,flow_mc,0,10;odor "odor" true true false 20 Text 0 0,First,#,Springs_Project,odor,0,20;water_odor "water_odor" true true false 250 Text 0 0,First,#,Springs_Project,water_odor,0,250;fish_seen "fish_seen" true true false 1 Text 0 0,First,#,Springs_Project,fish_seen,0,1;amphipods_ "amphipods_" true true false 1 Text 0 0,First,#,Springs_Project,amphipods_,0,1;plants "plants" true true false 20 Text 0 0,First,#,Springs_Project,plants,0,20;cryptogams "cryptogams" true true false 20 Text 0 0,First,#,Springs_Project,cryptogams,0,20;temp_c "temp_c" true true false 8 Double 0 0,First,#,Springs_Project,temp_c,-1,-1;temp_mc "temp_mc" true true false 10 Text 0 0,First,#,Springs_Project,temp_mc,0,10;cond "cond" true true false 8 Double 0 0,First,#,Springs_Project,cond,-1,-1;cond_mc "cond_mc" true true false 10 Text 0 0,First,#,Springs_Project,cond_mc,0,10;ph "ph" true true false 8 Double 0 0,First,#,Springs_Project,ph,-1,-1;ph_mc "ph_mc" true true false 10 Text 0 0,First,#,Springs_Project,ph_mc,0,10;eh "eh" true true false 8 Double 0 0,First,#,Springs_Project,eh,-1,-1;eh_mc "eh_mc" true true false 10 Text 0 0,First,#,Springs_Project,eh_mc,0,10;turb "turb" true true false 10 Text 0 0,First,#,Springs_Project,turb,0,10;turb_mc "turb_mc" true true false 10 Text 0 0,First,#,Springs_Project,turb_mc,0,10;do_ "do_" true true false 8 Double 0 0,First,#,Springs_Project,do_,-1,-1;do_mc "do_mc" true true false 10 Text 0 0,First,#,Springs_Project,do_mc,0,10;remarks "remarks" true true false 254 Text 0 0,First,#,Springs_Project,remarks,0,254;mssid "mssid" true true false 11 Text 0 0,First,#,Springs_Project,mssid,0,11;relateid "relateid" true true false 10 Text 0 0,First,#,Springs_Project,relateid,0,10;feat_label "feat_label" true true false 6 Text 0 0,First,#,Springs_Project,feat_label,0,6;status "status" true true false 25 Text 0 0,First,#,Springs_Project,status,0,25;certifier "certifier" true true false 150 Text 0 0,First,#,Springs_Project,certifier,0,150;date_certi "date_certi" true true false 8 Date 0 0,First,#,Springs_Project,date_certi,-1,-1;aquifer "aquifer" true true false 10 Text 0 0,First,#,Springs_Project,aquifer,0,10;aquifer_mc "aquifer_mc" true true false 20 Text 0 0,First,#,Springs_Project,aquifer_mc,0,20;county_c "county_c" true true false 2 Text 0 0,First,#,Springs_Project,county_c,0,2;township "township" true true false 4 Long 0 0,First,#,Springs_Project,township,-1,-1;range "range" true true false 4 Long 0 0,First,#,Springs_Project,range,-1,-1;range_dir "range_dir" true true false 1 Text 0 0,First,#,Springs_Project,range_dir,0,1;section "section" true true false 4 Long 0 0,First,#,Springs_Project,section,-1,-1;subsection "subsection" true true false 6 Text 0 0,First,#,Springs_Project,subsection,0,6;mgsquad_c "mgsquad_c" true true false 4 Text 0 0,First,#,Springs_Project,mgsquad_c,0,4;elevation "elevation" true true false 8 Double 0 0,First,#,Springs_Project,elevation,-1,-1;elev_mc "elev_mc" true true false 3 Text 0 0,First,#,Springs_Project,elev_mc,0,3;deposittyp "deposittyp" true true false 4 Text 0 0,First,#,Springs_Project,deposittyp,0,4;depth2bdrk "depth2bdrk" true true false 8 Double 0 0,First,#,Springs_Project,depth2bdrk,-1,-1;first_bdrk "first_bdrk" true true false 4 Text 0 0,First,#,Springs_Project,first_bdrk,0,4;form_top "form_top" true true false 4 Text 0 0,First,#,Springs_Project,form_top,0,4;form_bot "form_bot" true true false 4 Text 0 0,First,#,Springs_Project,form_bot,0,4;loc_mc "loc_mc" true true false 3 Text 0 0,First,#,Springs_Project,loc_mc,0,3;loc_src "loc_src" true true false 5 Text 0 0,First,#,Springs_Project,loc_src,0,5;loc_date "loc_date" true true false 4 Long 0 0,First,#,Springs_Project,loc_date,-1,-1;loc_date_f "loc_date_f" true true false 8 Date 0 0,First,#,Springs_Project,loc_date_f,-1,-1;geoc_src "geoc_src" true true false 5 Text 0 0,First,#,Springs_Project,geoc_src,0,5;geoc_date "geoc_date" true true false 4 Long 0 0,First,#,Springs_Project,geoc_date,-1,-1;geoc_date_ "geoc_date_" true true false 8 Date 0 0,First,#,Springs_Project,geoc_date_,-1,-1;geoc_by "geoc_by" true true false 5 Text 0 0,First,#,Springs_Project,geoc_by,0,5;utme "utme" true true false 4 Long 0 0,First,#,Springs_Project,utme,-1,-1;utmn "utmn" true true false 4 Long 0 0,First,#,Springs_Project,utmn,-1,-1;utm_accura "utm_accura" true true false 1 Text 0 0,First,#,Springs_Project,utm_accura,0,1;utm_zone "utm_zone" true true false 4 Long 0 0,First,#,Springs_Project,utm_zone,-1,-1;datum "datum" true true false 5 Text 0 0,First,#,Springs_Project,datum,0,5;file_src "file_src" true true false 30 Text 0 0,First,#,Springs_Project,file_src,0,30;dataset "dataset" true true false 3 Text 0 0,First,#,Springs_Project,dataset,0,3;entry_date "entry_date" true true false 4 Long 0 0,First,#,Springs_Project,entry_date,-1,-1;entry_da_1 "entry_da_1" true true false 8 Date 0 0,First,#,Springs_Project,entry_da_1,-1,-1;entry_by "entry_by" true true false 5 Text 0 0,First,#,Springs_Project,entry_by,0,5;updt_date "updt_date" true true false 4 Long 0 0,First,#,Springs_Project,updt_date,-1,-1;updt_date_ "updt_date_" true true false 8 Date 0 0,First,#,Springs_Project,updt_date_,-1,-1;updt_by "updt_by" true true false 5 Text 0 0,First,#,Springs_Project,updt_by,0,5;field_chk_ "field_chk_" true true false 4 Long 0 0,First,#,Springs_Project,field_chk_,-1,-1;created_us "created_us" true true false 254 Text 0 0,First,#,Springs_Project,created_us,0,254;created_da "created_da" true true false 8 Date 0 0,First,#,Springs_Project,created_da,-1,-1;last_edite "last_edite" true true false 254 Text 0 0,First,#,Springs_Project,last_edite,0,254;last_edi_1 "last_edi_1" true true false 8 Date 0 0,First,#,Springs_Project,last_edi_1,-1,-1;plant_comm "plant_comm" true true false 100 Text 0 0,First,#,Springs_Project,plant_comm,0,100;creator_na "creator_na" true true false 100 Text 0 0,First,#,Springs_Project,creator_na,0,100;creator_da "creator_da" true true false 8 Date 0 0,First,#,Springs_Project,creator_da,-1,-1;editor_nam "editor_nam" true true false 100 Text 0 0,First,#,Springs_Project,editor_nam,0,100;editor_dat "editor_dat" true true false 8 Date 0 0,First,#,Springs_Project,editor_dat,-1,-1;name_offic "name_offic" true true false 125 Text 0 0,First,#,Springs_Project,name_offic,0,125;contact_na "contact_na" true true false 150 Text 0 0,First,#,Springs_Project,contact_na,0,150;email_addr "email_addr" true true false 150 Text 0 0,First,#,Springs_Project,email_addr,0,150;phone "phone" true true false 25 Text 0 0,First,#,Springs_Project,phone,0,25;mdh_id "mdh_id" true true false 50 Text 0 0,First,#,Springs_Project,mdh_id,0,50;bank_relat "bank_relat" true true false 25 Text 0 0,First,#,Springs_Project,bank_relat,0,25;spring_run "spring_run" true true false 25 Text 0 0,First,#,Springs_Project,spring_run,0,25;multi_samp "multi_samp" true true false 4 Long 0 0,First,#,Springs_Project,multi_samp,-1,-1;sample_no "sample_no" true true false 4 Long 0 0,First,#,Springs_Project,sample_no,-1,-1;nitrate "nitrate" true true false 10 Text 0 0,First,#,Springs_Project,nitrate,0,10;no3_mc "no3_mc" true true false 10 Text 0 0,First,#,Springs_Project,no3_mc,0,10;measure_re "measure_re" true true false 250 Text 0 0,First,#,Springs_Project,measure_re,0,250;meas_src "meas_src" true true false 10 Text 0 0,First,#,Springs_Project,meas_src,0,10;measure_da "measure_da" true true false 8 Date 0 0,First,#,Springs_Project,measure_da,-1,-1;meas_date "meas_date" true true false 4 Long 0 0,First,#,Springs_Project,meas_date,-1,-1;remarks_lo "remarks_lo" true true false 254 Text 0 0,First,#,Springs_Project,remarks_lo,0,254;AREA "AREA" true true false 8 Double 0 0,First,#,LandType_Project,AREA,-1,-1;PERIMETER "PERIMETER" true true false 8 Double 0 0,First,#,LandType_Project,PERIMETER,-1,-1;ECSLTPY2_ "ECSLTPY2_" true true false 8 Double 0 0,First,#,LandType_Project,ECSLTPY2_,-1,-1;ECSLTPY2_I "ECSLTPY2_I" true true false 8 Double 0 0,First,#,LandType_Project,ECSLTPY2_I,-1,-1;PROVNAME "PROVNAME" true true false 35 Text 0 0,First,#,LandType_Project,PROVNAME,0,35;ECS_PROV "ECS_PROV" true true false 3 Text 0 0,First,#,LandType_Project,ECS_PROV,0,3;SECNAME "SECNAME" true true false 35 Text 0 0,First,#,LandType_Project,SECNAME,0,35;ECS_SEC "ECS_SEC" true true false 4 Text 0 0,First,#,LandType_Project,ECS_SEC,0,4;SUBSECNAME "SUBSECNAME" true true false 40 Text 0 0,First,#,LandType_Project,SUBSECNAME,0,40;ECS_SUBSEC "ECS_SUBSEC" true true false 5 Text 0 0,First,#,LandType_Project,ECS_SUBSEC,0,5;LTANAME "LTANAME" true true false 40 Text 0 0,First,#,LandType_Project,LTANAME,0,40;ECS_LTA "ECS_LTA" true true false 10 Text 0 0,First,#,LandType_Project,ECS_LTA,0,10;VERSION "VERSION" true true false 5 Text 0 0,First,#,LandType_Project,VERSION,0,5;Shape_Leng "Shape_Leng" true true false 8 Double 0 0,First,#,LandType_Project,Shape_Leng,-1,-1;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,LandType_Project,Shape_Length,-1,-1;Shape_Area "Shape_Area" false true true 8 Double 0 0,First,#,LandType_Project,Shape_Area,-1,-1', "INTERSECT", None, '')

<Result 'E:\\ArcGIS_1\\Lab1\\Lab1_API\\Springs_SpatialJoin_LandType.shp'>

In [5]:
# Reads the first 5 rows of the attribute table of the spatial join output as a geoDataFrame
springs_landType_table = gpd.read_file(r'E:\ArcGIS_1\Lab1\Lab1_API\Springs_SpatialJoin_LandType.shp')
springs_landType_table.head()

,Join_Count,TARGET_FID,survey_nam,name,feature,feature_su,feature_ar,spring_typ,spring_t_1,lithology,lith_comme,mineral_pr,min_prec_c,photo,flow_measu,field_meas,chemistry_,field_chec,field_ch_1,gcm_code,general_co,flowing,flow,flow_units,flow_mc,odor,water_odor,fish_seen,amphipods_,plants,cryptogams,temp_c,temp_mc,cond,cond_mc,ph,ph_mc,eh,eh_mc,turb,...,last_edite,last_edi_1,plant_comm,creator_na,creator_da,editor_nam,editor_dat,name_offic,contact_na,email_addr,phone,mdh_id,bank_relat,spring_run,multi_samp,sample_no,nitrate,no3_mc,measure_re,meas_src,measure_da,meas_date,remarks_lo,AREA,PERIMETER,ECSLTPY2_,ECSLTPY2_I,PROVNAME,ECS_PROV,SECNAME,ECS_SEC,SUBSECNAME,ECS_SUBSEC,LTANAME,ECS_LTA,VERSION,Shape_Leng,Shape_Le_1,Shape_Area,geometry
0,1,0,None,Lafky Springs,A,None,None,None,None,None,None,None,None,None,None,None,None,Y,2017-05-04,J,None,None,0.0,None,None,None,None,None,None,None,None,0.0,None,0.0,None,0.0,None,0.0,None,None,...,gisproxy,2017-06-13,None,None,None,Calvin Alexander,2017-06-13,None,Daryl Lafky,None,None,None,None,None,1,0,None,None,None,None,None,0,None,1.335477e+07,7.272774e+04,482.0,481.0,Eastern Broadleaf Forest Province,222,Paleozoic Plateau,222L,The Blufflands,222Lc,Alluvial Plain,222Lc01,2000a,7.272774e+04,0.0,1.335477e+07,POINT (598628.410 4872621.874)
1,1,1,None,Bear Overflow Spring,A,spring,single,other,Ephemeral overflow spring that resurges water ...,None,None,None,None,None,None,None,None,Y,2015-04-02,L1,None,None,0.0,None,None,None,None,None,None,None,None,0.0,None,0.0,None,0.0,None,0.0,None,None,...,gisproxy,2017-06-16,None,None,None,Calvin Alexander,2017-06-16,None,None,None,None,None,None,None,0,0,None,None,None,None,None,0,This is the overflow spring for MN55:A00406. I...,1.691159e+09,1.238136e+06,481.0,480.0,Eastern Broadleaf Forest Province,222,Paleozoic Plateau,222L,Rochester Plateau,222Lf,Stewartville Plain,222Lf03,2000a,1.238136e+06,0.0,1.691159e+09,POINT (557688.404 4869553.874)
2,1,2,None,Curtain Creek Rise,S,spring,single,fluvial,None,None,None,none,None,None,None,None,None,Y,2018-08-03,None,None,None,0.0,None,None,None,None,None,None,None,None,0.0,None,0.0,None,0.0,None,0.0,None,None,...,gisproxy,2018-08-06,None,None,None,Gregory Brick,2018-08-06,None,None,None,None,None,None,None,0,0,None,None,None,None,None,0,There are 2 springs here--where the water eman...,3.223651e+06,1.171417e+04,368.0,367.0,Laurentian Mixed Forest Province,212,Southern Superior Uplands,212J,St. Croix Moraine,212Jd,Polk Basalt Moraines,212Jd05,2000a,1.171417e+04,0.0,3.223651e+06,POINT (525707.398 5026883.897)
3,1,3,None,Spring,A,spring,cluster,None,None,None,None,None,None,None,None,None,None,Y,2003-03-13,A,None,None,0.0,None,None,None,None,None,None,None,None,0.0,None,0.0,None,0.0,None,0.0,None,None,...,gisproxy,2017-06-16,None,None,None,Calvin Alexander,2017-06-16,None,STEVE BAILEY,None,None,None,None,None,1,0,None,None,None,None,None,0,None,8.472077e+06,5.705909e+04,504.0,503.0,Eastern Broadleaf Forest Province,222,Paleozoic Plateau,222L,The Blufflands,222Lc,Elba Slopes,222Lc11,2000a,5.705909e+04,0.0,8.472077e+06,POINT (558804.404 4860096.872)
4,1,4,None,Swanson Spring #2,A,None,None,None,None,None,None,None,None,None,None,None,None,Y,2015-12-07,L1,None,None,35.0,None,None,None,None,None,None,None,None,8.0,None,0.0,None,0.0,None,0.0,None,None,...,gisproxy,2017-06-16,None,None,None,Calvin Alexander,2017-06-16,None,WALTER S. SWANSON,None,None,None,None,None,1,1,20.8,None,"SPRING IS CLEAN, OUTCROP.",OCHD,1986-11-05,19861105,"1986, Geri Maki, Olmsted Co. Spring is cl...",8.472077e+06,5.705909e+04,504.0,503.0,Eastern Broadleaf Forest Province,222,Paleozoic Plateau,222L,The Blufflands,222Lc,Elba Slopes,222Lc11,2000a,5.705909e+04,0.0,8.472077e+06,POINT (558150.404 4860576.872)


## Google

Features are created with the geographic information obtained from Google Places on The University of Minnesota - Minneapolis, and The Huntington Bank Stadium.

In [6]:
# Dictionaries are created with the information retrieved from Google Places

# University of Minnesota
university_link = r'https://maps.googleapis.com/maps/api/place/findplacefromtext/json?fields=formatted_address%2Cname%2Crating%2Copening_hours%2Cgeometry&input=University%20of%20Minnesota%20Minneapolis&inputtype=textquery&key=YOUR_API_KEY'
university = requests.get(university_link)
university_dic = json.loads(university.text)

# The Huntington Bank Stadium
Stadium_link = r'https://maps.googleapis.com/maps/api/place/findplacefromtext/json?fields=formatted_address%2Cname%2Crating%2Copening_hours%2Cgeometry&input=The%20Huntington%20Bank%20Stadium%20Minneapolis&inputtype=textquery&key=YOUR_API_KEY'
stadium = requests.get(Stadium_link)
stadium_dic = json.loads(stadium.text)

In [7]:
# The coordinates for each dataset are extracted from the dictionaries and stored in lists

def createListWithPoints(dictionary):
    coords = dictionary['candidates'][0]['geometry']['viewport']
    point1 = [float(coords['northeast']['lng']), float(coords['northeast']['lat'])]
    point2 = [float(coords['northeast']['lng']), float(coords['southwest']['lat'])]
    point3 = [float(coords['southwest']['lng']), float(coords['southwest']['lat'])]
    point4 = [float(coords['southwest']['lng']), float(coords['northeast']['lat'])]
    point_list = [point1, point2, point3, point4]
    return point_list

university_points = createListWithPoints(university_dic)
stadium_points = createListWithPoints(stadium_dic)

The following cell was created based on the code shown in https://www.youtube.com/watch?v=fxUagyDxDGs and https://gis.stackexchange.com/questions/97545/using-fiona-to-write-a-new-shapefile-from-scratch

In [8]:
# Creates shapefiles with the coordinates from the lists

umn_schema =  {'geometry': 'Point', 'properties': {'location': 'str'}}

with fiona.open(r"E:\ArcGIS_1\Lab1\Lab1_API\umn.shp", 'w', crs = from_epsg(4326), driver = 'ESRI Shapefile', schema = umn_schema) as output:
    for i in range(len(university_points)):
          # geometry
          point = geometry.Point((university_points[i][0]), university_points[i][1])
          # attributes
          prop = {'location': 'umn'}
          # write the row (geometry + attributes in GeoJSON format)
          output.write({'geometry': geometry.mapping(point), 'properties':prop})
        
with fiona.open(r"E:\ArcGIS_1\Lab1\Lab1_API\stadium.shp", 'w', crs = from_epsg(4326), driver = 'ESRI Shapefile', schema = umn_schema) as output:
    for i in range(len(stadium_points)):
          # geometry
          point = geometry.Point((stadium_points[i][0]), stadium_points[i][1])
          # attributes
          prop = {'location': 'The Huntington Stadium'}
          # write the row (geometry + attributes in GeoJSON format)
          output.write({'geometry': geometry.mapping(point), 'properties':prop})

In [9]:
# Transforms the points into polygons, adds the location in the attribute table, and projects the coordinate system to WGS 1984 UTM Zone 15N

# University of Minnesota
arcpy.management.MinimumBoundingGeometry(r"E:\ArcGIS_1\Lab1\Lab1_API\umn.shp", r"E:\ArcGIS_1\Lab1\Lab1_API\university_poly.shp", "RECTANGLE_BY_AREA", "ALL", None, "NO_MBG_FIELDS")
arcpy.management.AddField(r"E:\ArcGIS_1\Lab1\Lab1_API\university_poly.shp", "location", "TEXT", None, None, 50, '', "NULLABLE", "NON_REQUIRED", '')
arcpy.management.CalculateField(r"E:\ArcGIS_1\Lab1\Lab1_API\university_poly.shp", "location", "'University of Minnesota Minneapolis'", "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")
arcpy.management.Project(r"E:\ArcGIS_1\Lab1\Lab1_API\university_poly.shp", r"E:\ArcGIS_1\Lab1\Lab1_API\university_project.shp", 'PROJCS["WGS_1984_UTM_Zone_15N",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-93.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]', None, 'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', "NO_PRESERVE_SHAPE", None, "NO_VERTICAL")

# The Huntington Bank Stadium
arcpy.management.MinimumBoundingGeometry(r"E:\ArcGIS_1\Lab1\Lab1_API\stadium.shp", r"E:\ArcGIS_1\Lab1\Lab1_API\stadium_poly.shp", "RECTANGLE_BY_AREA", "ALL", None, "NO_MBG_FIELDS")
arcpy.management.AddField(r"E:\ArcGIS_1\Lab1\Lab1_API\stadium_poly.shp", "location", "TEXT", None, None, 50, '', "NULLABLE", "NON_REQUIRED", '')
arcpy.management.CalculateField(r"E:\ArcGIS_1\Lab1\Lab1_API\stadium_poly.shp", "location", "'The Huntington Bank Stadium'", "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")
arcpy.management.Project(r"E:\ArcGIS_1\Lab1\Lab1_API\stadium_poly.shp", r"E:\ArcGIS_1\Lab1\Lab1_API\stadium_project.shp", 'PROJCS["WGS_1984_UTM_Zone_15N",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-93.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]', None, 'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', "NO_PRESERVE_SHAPE", None, "NO_VERTICAL")

<Result 'E:\\ArcGIS_1\\Lab1\\Lab1_API\\stadium_project.shp'>

In [10]:
# Spatial join
arcpy.analysis.SpatialJoin(r"E:\ArcGIS_1\Lab1\Lab1_API\stadium_project.shp", r"E:\ArcGIS_1\Lab1\Lab1_API\university_project.shp", r"E:\ArcGIS_1\Lab1\Lab1_API\stadium_SpatialJoin_university.shp", "JOIN_ONE_TO_ONE", "KEEP_ALL", 'location "location" true true false 50 Text 0 0,First,#,stadium_poly,location,0,50;location_1 "location" true true false 50 Text 0 0,First,#,university_poly,location,0,50', "INTERSECT", None, '')

<Result 'E:\\ArcGIS_1\\Lab1\\Lab1_API\\stadium_SpatialJoin_university.shp'>

In [11]:
# Reads the attribute table of the spatial join output as a geoDataFrame
stadium_university_table = gpd.read_file(r"E:\ArcGIS_1\Lab1\Lab1_API\stadium_SpatialJoin_university.shp")
stadium_university_table.head()

,Join_Count,TARGET_FID,location,location_1,geometry
0,1,0,The Huntington Bank Stadium,University of Minnesota Minneapolis,"POLYGON ((481996.318 4980753.867, 482658.391 4..."


## NDAWN
Information about the weekly temperature (degrees Fahrenheit) and total solar radiation (langleys) recorded by the stations Ada and Adams is downloaded from the North Dakota Agricultural Weather Network Center

In [12]:
# Retrieves the CSV files from the NDAWN portal

# Temperature
temp_url = r'https://ndawn.ndsu.nodak.edu/table.csv?station=78&station=111&variable=wdavt&ttype=weekly&quick_pick=&begin_date=2022-09-23&count=1'
retrieve(temp_url, 'weekly_temp.csv')
temp_df = pd.read_csv("weekly_temp.csv", skiprows = [0, 1, 2, 4])

# Solar radiation
radiation_url = r'https://ndawn.ndsu.nodak.edu/table.csv?station=78&station=111&variable=wdsr&ttype=weekly&quick_pick=&begin_date=2022-09-23&count=1'
retrieve(radiation_url, 'weekly_radiation.csv')
radiation_df = pd.read_csv("weekly_radiation.csv", skiprows = [0, 1, 2, 4])

In [13]:
# Creates the point shapefiles

temp_schema = {'geometry': 'Point', 'properties': {'Station Name': 'str', 'Avg Temp': 'float'}}
radiation_schema = {'geometry': 'Point', 'properties': {'Station Name': 'str', 'Total Solar Rad': 'float'}}

with fiona.open(r"E:\ArcGIS_1\Lab1\Lab1_API\temp.shp", 'w', crs = from_epsg(4326), driver = 'ESRI Shapefile', schema = temp_schema) as output:
    for i in range(len(temp_df)):
          # geometry
          point = geometry.Point(temp_df.loc[i, 'Longitude'], temp_df.loc[i, 'Latitude'])
          # attributes
          prop = {'Station Name': temp_df.loc[i, 'Station Name'], 'Avg Temp': temp_df.loc[i, 'Avg Temp']}
          # write the row (geometry + attributes in GeoJSON format)
          output.write({'geometry': geometry.mapping(point), 'properties':prop})
        
with fiona.open(r"E:\ArcGIS_1\Lab1\Lab1_API\radiation.shp", 'w', crs = from_epsg(4326), driver = 'ESRI Shapefile', schema = radiation_schema) as output:
    for i in range(len(radiation_df)):
          # geometry
          point = geometry.Point(radiation_df.loc[i, 'Longitude'], radiation_df.loc[i, 'Latitude'])
          # attributes
          prop = {'Station Name': radiation_df.loc[i, 'Station Name'], 'Total Solar Rad': radiation_df.loc[i, 'Total Solar Rad']}
          # write the row (geometry + attributes in GeoJSON format)
          output.write({'geometry': geometry.mapping(point), 'properties':prop})

In [14]:
# The shapefiles are projected to WGS 1984 UTM Zone 15N and the spatial join is created
arcpy.management.Project(r"E:\ArcGIS_1\Lab1\Lab1_API\temp.shp", r"E:\ArcGIS_1\Lab1\Lab1_API\temp_project.shp", 'PROJCS["WGS_1984_UTM_Zone_15N",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-93.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]', None, 'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', "NO_PRESERVE_SHAPE", None, "NO_VERTICAL")
arcpy.management.Project(r"E:\ArcGIS_1\Lab1\Lab1_API\radiation.shp", r"E:\ArcGIS_1\Lab1\Lab1_API\radiation_project.shp", 'PROJCS["WGS_1984_UTM_Zone_15N",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-93.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]', None, 'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', "NO_PRESERVE_SHAPE", None, "NO_VERTICAL")
arcpy.analysis.SpatialJoin(r"E:\ArcGIS_1\Lab1\Lab1_API\temp_project.shp", r"E:\ArcGIS_1\Lab1\Lab1_API\radiation_project.shp", r"E:\ArcGIS_1\Lab1\Lab1_API\weather_SpatialJoin.shp", "JOIN_ONE_TO_ONE", "KEEP_COMMON", 'Station_Na "Station_Na" true true false 80 Text 0 0,First,#,temp_project,Station_Na,0,80;Avg_Temp "Avg_Temp" true true false 19 Double 15 18,First,#,temp_project,Avg_Temp,-1,-1;Total_Sola "Total_Sola" true true false 19 Double 15 18,First,#,radiation_project,Total_Sola,-1,-1', "INTERSECT", None, '')

<Result 'E:\\ArcGIS_1\\Lab1\\Lab1_API\\weather_SpatialJoin.shp'>

In [15]:
# Reads the attribute table of the spatial join output as a geoDataFrame
weather_table = gpd.read_file(r"E:\ArcGIS_1\Lab1\Lab1_API\weather_SpatialJoin.shp")
weather_table.head()

,Join_Count,TARGET_FID,Station_Na,Avg_Temp,Total_Sola,geometry
0,1,0,Ada,53.126,287.386,POINT (234460.130 5246847.894)
1,1,1,Adams,52.223,333.649,POINT (125094.217 5384315.197)


## GDB

In [16]:
# Creates a geodatabase to store the integrated datasets 
arcpy.management.CreateFileGDB(r"E:\ArcGIS_1\Lab1\Lab1_API", "Final_geodatabase", "CURRENT")
arcpy.conversion.FeatureClassToGeodatabase("E:\ArcGIS_1\Lab1\Lab1_API\Springs_SpatialJoin_LandType.shp;E:\ArcGIS_1\Lab1\Lab1_API\stadium_SpatialJoin_university.shp;E:\ArcGIS_1\Lab1\Lab1_API\weather_SpatialJoin.shp", r"E:\ArcGIS_1\Lab1\Lab1_API\Final_geodatabase.gdb")

<Result 'E:\\ArcGIS_1\\Lab1\\Lab1_API\\Final_geodatabase.gdb'>